In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_session.region_name

import time
cur_time=time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
base_job_name='faker-'+cur_time

In [ ]:
bucket=sagemaker.Session().default_bucket()
checkpoint_in_bucket="checkpoints"
checkpoint_s3_bucket="s3://{}/{}/{}".format(bucket, base_job_name, checkpoint_in_bucket)
checkpoint_dir="/opt/ml/checkpoints"

hyperparameters = {'work_dir': '/opt/ml',
                   'data_name': 'train.txt',
                   'seq_len': 64,
                   'module': 'lstmLN',
                   'hidden_dim': 32,
                   'num_layer': 3,
                   'iteration': 20000,
                   'batch_size': 1024,
                   'metric_iteration': 5,
                   'learning_rate': 1e-4,
                  }

estimator = TensorFlow(entry_point='run.sh',
                       source_dir='./',
                       role='',
                       framework_version='1.15',
                       py_version='py37',
                       use_spot_instances=True,
                       max_wait=5*86400,
                       max_run=4*86400,
                       tags=[{"Key":"CodeName","Value":"ctr"}],
                       instance_type='ml.p3.2xlarge',
                       instance_count=1,
                       volume_size=40,
                       hyperparameters=hyperparameters,
                       script_mode=True,
                       disable_profiler=True,
                       base_job_name=base_job_name,
                       checkpoint_s3_uri=checkpoint_s3_bucket,
                       checkpoint_local_path=checkpoint_dir
                       )

In [ ]:
inputs='s3://algo.yeahmobi.com/faker/demo/'
estimator.fit(inputs={'train': inputs},
              wait=False,
              job_name=base_job_name)
